In [3]:
import subprocess
import shutil
import torch
import torchvision
from torchvision import transforms
import pickle

/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = torchvision.datasets.ImageFolder(root='dataset/train', transform=preprocess)
test_data = torchvision.datasets.ImageFolder(root='dataset/test', transform=preprocess)


In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=18, shuffle=True, num_workers=4)

In [7]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.to(device)
model_name = 'inception'

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.3)

In [9]:
test_accs = dict()

for epoch in range(8):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if i % 2000 == 1:
            correct = 0
            total = 0
            with torch.no_grad():
                for ix, data in enumerate(test_loader):
                    if ix > 100:
                        break
                    images, labels = data
                    images, labels = images.cuda(), labels.cuda()
                    # calculate outputs by running images through the network
                    outputs = model(images)
                    # the class with the highest energy is what we choose as prediction
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            acc = correct / total
            test_accs[(epoch, i)] = acc
            print(f'Accuracy of the network on the test images: {100*acc} %')
            with open(f'accuracies_{model_name}.pickle', 'wb') as fp:
                pickle.dump(test_accs, fp)
        if epoch == 0 and (i % 500 == 1):
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')

        if epoch > 0 and i % 5000 == 0:
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
    
    scheduler.step()

print('Finished Training')

Accuracy of the network on the test images: 36.19361936193619 %
[1,   200] loss: 0.835
[1,   400] loss: 0.474
[1,   600] loss: 0.381
[1,   800] loss: 0.334
[1,  1000] loss: 0.286
[1,  1200] loss: 0.279
[1,  1400] loss: 0.240
[1,  1600] loss: 0.217
[1,  1800] loss: 0.198
[1,  2000] loss: 0.208
Accuracy of the network on the test images: 81.84818481848185 %
[1,  2200] loss: 0.203
[1,  2400] loss: 0.172
[1,  2600] loss: 0.159
[1,  2800] loss: 0.168
[1,  3000] loss: 0.159
[1,  3200] loss: 0.161
[1,  3400] loss: 0.155
[1,  3600] loss: 0.147
[1,  3800] loss: 0.133
[1,  4000] loss: 0.131
Accuracy of the network on the test images: 82.23322332233224 %
[1,  4200] loss: 0.126
[1,  4400] loss: 0.124
[1,  4600] loss: 0.133
[1,  4800] loss: 0.139
[1,  5000] loss: 0.113
[1,  5200] loss: 0.115
[1,  5400] loss: 0.119
[1,  5600] loss: 0.097
[1,  5800] loss: 0.112
Accuracy of the network on the test images: 83.4983498349835 %
[2,   200] loss: 0.068
[2,   400] loss: 0.049
[2,   600] loss: 0.041
[2,   800

In [12]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
confusion_matrix = torch.zeros(4, 4)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
print(confusion_matrix.diag()/confusion_matrix.sum(1))

Accuracy of the network on the test images: 88 %
tensor([0.8449, 0.9488, 0.8644, 0.9050])


In [15]:
!ls

 10_Captum_conventional.ipynb
 1_ECCV16_Dataset_Locate_the_towns_amog_tars.ipynb
 2_Dataset_exploration.ipynb
 3_ECCV16_Dataset_Procedure_for_dataset_split.ipynb
 4_ECCV16_Dataset_First_simple_training_run.ipynb
 5_ECCV16_Dataset_Lucent.ipynb
 5_Lucent_Fresh_new_network.ipynb
 6_5_Visualize_kitty.ipynb
'6_ECCV16_Small boi training.ipynb'
 6_Kitty_training.ipynb
 7_Longcat_training.ipynb
 8_InceptionNet_training.ipynb
 9_Lucent_Pretrained_GoogLeNet.ipynb
 __pycache__
 accuracies_inception.pickle
 accuracies_kitty.pickle
 accuracies_longcat.pickle
 captum_old
 captum_optim
 cities.py
 dataset
 eccv16_dataset.ipynb
 map.png
 prune_down.py
 remove_pairless.py
 saved_models
 train_test_split.py
 unzip_and_prune.py
